# Load Forecasting

## 부하 예측을 위한 데이터 수집 및 변환

### 부하 데이터 프로세싱

In [39]:
import pandas as pd

### 부하 데이터 불러오기

In [18]:
raw_gen_data = pd.read_excel('1.Data/raw_generation_data.xlsx')
raw_gen_data.head()

,Date,00,01,02,03,04,05,06,07,08,...,14,15,16,17,18,19,20,21,22,23
0,2018-01-01,62446,59691,57231,55500,54821,55000,55641,56212,55816,...,52206,52503,53553,55466,58560,60360,60588,60408,60333,61328
1,2018-01-02,62145,59313,56977,55828,55675,56392,58196,61468,66824,...,76154,77071,77019,77821,77781,75982,73583,71269,69243,68858
2,2018-01-03,70214,66714,64267,62935,62446,63043,64539,67464,72310,...,77396,78613,78719,80122,80242,78515,76217,73977,71968,71338
3,2018-01-04,72583,68811,66239,64792,64326,64771,66142,68994,73706,...,79590,80954,81231,82344,81784,79743,76962,74045,71729,71174
4,2018-01-05,72427,68787,66141,64752,64128,64570,65695,68262,72565,...,76755,77557,77869,79366,79199,77443,74884,72530,70450,69942


### stack을 활용하여 열을 행으로 변경

### **pandas.DataFrame.stack**

pandas의 stack 함수는 데이터의 구조를 재조정하는 데 사용되는 매우 유용한 도구입니다. 이 함수는 DataFrame의 컬럼들을 로우로 "쌓아올려" MultiIndex를 가진 Series로 변환합니다. 이 과정을 이해하기 위해 간단한 예제를 살펴보겠습니다.

**예제1**

다음과 같은 DataFrame을 가지고 있다고 가정해봅시다:

|   | A | B |
|---|---|---|
| 0 | 1 | 4 |
| 1 | 2 | 5 |
| 2 | 3 | 6 |

이 DataFrame에 **'stack'** 함수를 적용하면 결과는 아래와 같습니다.

|   |   |   |
|---|---|---|
| 0 | A | 1 |
| 0 | B | 4 |
| 1 | A | 2 |
| 1 | B | 5 |
| 2 | A | 3 |
| 2 | B | 6 |

여기서 주목해야 할 점은 결과가 MultiIndex를 가진 Series임을 확인할 수 있습니다. 첫 번째 레벨의 인덱스는 원본 DataFrame의 로우 인덱스이며, 두 번째 레벨의 인덱스는 원본 DataFrame의 컬럼입니다.

**예제2**

다음과 같은 다중 레벨 컬럼을 가진 DataFrame을 고려해봅시다:

|   |     |  A  |  A  |  B  |  B  |
|---|-----|-----|-----|-----|-----|
|   |     | X   | Y   | X   | Y   |
|---|-----|-----|-----|-----|-----|
| 0 |     | 1   | 2   | 3   | 4   |
| 1 |     | 5   | 6   | 7   | 8   |


이 경우, level 매개변수를 사용하여 어떤 레벨을 로우로 쌓을지 지정할 수 있습니다. 예를 들어, 최상위 레벨을 로우로 쌓아보겠습니다:

stacked_level_0 = df.stack(level=0)

결과는:

|   |   | X | Y |
|---|---|---|---|
| 0 | A | 1 | 2 |
| 0 | B | 3 | 4 |
| 1 | A | 5 | 6 |
| 1 | B | 7 | 8 |

stack 함수는 데이터를 재구조화하는 데 있어 강력한 도구로, 데이터 분석 및 변환 작업에서 자주 사용됩니다.

In [34]:
stacked_data = raw_gen_data.set_index('Date').stack().reset_index()
stacked_data

,Date,level_1,0
0,2018-01-01,00,62446
1,2018-01-01,01,59691
2,2018-01-01,02,57231
3,2018-01-01,03,55500
4,2018-01-01,04,54821
...,...,...,...
22627,2020-07-31,19,70347
22628,2020-07-31,20,70092
22629,2020-07-31,21,69176
22630,2020-07-31,22,66569


In [35]:
stacked_data.columns = ['Date', 'Hour', 'Generation']
stacked_data

,Date,Hour,Generation
0,2018-01-01,00,62446
1,2018-01-01,01,59691
2,2018-01-01,02,57231
3,2018-01-01,03,55500
4,2018-01-01,04,54821
...,...,...,...
22627,2020-07-31,19,70347
22628,2020-07-31,20,70092
22629,2020-07-31,21,69176
22630,2020-07-31,22,66569


### **pandas.DataFrame.datetime**

pandas는 날짜와 시간을 효과적으로 처리하기 위한 다양한 도구와 기능을 제공합니다. datetime은 Python의 기본 라이브러리 중 하나로, 날짜와 시간을 다루는 데 사용됩니다. pandas는 이 datetime 모듈을 기반으로 확장된 기능들을 제공하며, 특히 Timestamp 객체와 관련 메서드들을 통해 강력한 시계열 분석 기능을 지원합니다.

pandas의 pd.to_datetime 함수는 다양한 형식의 날짜와 시간 문자열을 pandas Timestamp 객체로 변환하는데 사용됩니다.

In [36]:
stacked_data['Time'] = pd.to_datetime(stacked_data['Date'].astype(str) + ' ' + stacked_data['Hour'].astype(str) + ':00:00')
stacked_data

,Date,Hour,Generation,Time
0,2018-01-01,00,62446,2018-01-01 00:00:00
1,2018-01-01,01,59691,2018-01-01 01:00:00
2,2018-01-01,02,57231,2018-01-01 02:00:00
3,2018-01-01,03,55500,2018-01-01 03:00:00
4,2018-01-01,04,54821,2018-01-01 04:00:00
...,...,...,...,...
22627,2020-07-31,19,70347,2020-07-31 19:00:00
22628,2020-07-31,20,70092,2020-07-31 20:00:00
22629,2020-07-31,21,69176,2020-07-31 21:00:00
22630,2020-07-31,22,66569,2020-07-31 22:00:00


In [37]:
final_data_using_stack = stacked_data[['Time', 'Generation']].sort_values(by='Time').reset_index(drop=True)
final_data_using_stack

,Time,Generation
0,2018-01-01 00:00:00,62446
1,2018-01-01 01:00:00,59691
2,2018-01-01 02:00:00,57231
3,2018-01-01 03:00:00,55500
4,2018-01-01 04:00:00,54821
...,...,...
22627,2020-07-31 19:00:00,70347
22628,2020-07-31 20:00:00,70092
22629,2020-07-31 21:00:00,69176
22630,2020-07-31 22:00:00,66569
